In [ ]:
# -*- coding: utf-8 -*-
from wmsourcingsdk import datalake
from wmsourcingsdk.unit_converter import UnitConverter
from wmsourcingsdk.datalake_publishing import DPCPublishing
import pandas as pd
from io import BytesIO

In [ ]:
# Get a list of new files
#files_updated = datalake.batch_files_recent('wood-mackenzie--corporate-analysis-reported-liquid-production--extract','wood-mackenzie--corporate-analysis-aggregated-reported-production--transform')
    

In [ ]:
            # If there are any new files
#if files_updated:  
# Process them in turn
#    for liquid_file in files_updated:     
x=0    
for liquid_file in datalake.list_batch_files('wood-mackenzie--corporate-analysis-reported-liquid-production--extract'): 

            # Assume we should always find an equivalent gas file with a similar name, and also prepare an output file
    gas_file = liquid_file.replace('liquid','gas')
#    print(gas_file + str(2))
    output_file = liquid_file.replace('reported-liquid-production','aggregated_production')
#    if liquid_file.startswith('reported_kufpecgas'):
#        continue

        # Load the liquids data from the Data Lake, and import the parquet file into a dataframe

    liquid_content = datalake.load_from_extract('wood-mackenzie--corporate-analysis-reported-liquid-production--extract', liquid_file)  
#    print(gas_file + str(3))
    df_liquid = pd.read_parquet(BytesIO(liquid_content))
#    print(gas_file + str(4))
        # Total by Company (should only ever be one) and by Year, and reset the index to flatten the column headings
    df_liquid = df_liquid.groupby(['Company Name','Year','Unit','HC_Type']).sum().reset_index()
#    print(gas_file + str(5))
        # Load the gas data from the Data Lake, and import the parquet file into a dataframe
    gas_content = datalake.load_from_extract('wood-mackenzie--corporate-analysis-reported-gas-production--extract', gas_file)  
#    print(gas_file + str(6))
    df_gas = pd.read_parquet(BytesIO(gas_content))
#    print(gas_file + str(7))
    if not df_gas.empty:
#        print(gas_file + str(8))
        df_gas = df_gas.groupby(['Company Name', 'Year', 'Unit', 'HC_Type']).sum().reset_index()
#        print(gas_file + str(9))
        df_gas['Valueboe'] = df_gas['Value'].apply(lambda x: x/6 if x>0 or x<0 else 0)
#        print(gas_file + str(10))
        df_gas_converted = df_gas[['Company Name','Year','Unit','HC_Type','Valueboe']].copy()
#        print(gas_file + str(11))
        df_gas_converted['Unit'] = 'mmboe'
#        print(gas_file + str(12))
        df_gas_converted.rename(columns={'Valueboe':'Value'}, inplace=True)

        # Total by Company (should only ever be one) and by Year, and reset the index to flatten the column headings
#    df_gas_converted = df_gas.groupby(['Company Name','Year','Unit','HC_Type']).sum().reset_index()

        # Create a combined dataframe with both liquids and gas
        df_total = pd.concat([df_liquid, df_gas_converted], ignore_index=True)
        df_total['Unit'] = 'kboe/d'
        df_total['HC_Type'] = 'Total'
        df_total = df_total.groupby(['Company Name','Year','Unit','HC_Type']).sum().reset_index()


        # Total by Company (should only ever be one) and by Year, and reset the index to flatten the column headings

        #df_total = df_total.groupby(['Company Name','Year','Unit','HC_Type']).sum().reset_index()      

        df_gas = df_gas[['Company Name','Year','Unit','HC_Type','Value']]

        # Combine all three datframes
        df_final = pd.concat([df_gas, df_liquid, df_total], ignore_index=True)
        df_final.insert(0, "Metric", 'Reported Production')
        df_final = df_final.groupby(['Company Name','Metric','Year','Unit','HC_Type']).sum().reset_index()
        df_final = df_final[['Company Name','Metric', 'HC_Type','Year','Unit','Value']].sort_values(by=['Company Name', 'Year', 'HC_Type'])


        df_final= df_final.loc[df_final['Year'] != 0]

        print(df_final)

        cnt=df_final['Company Name'].nunique()
        x+=cnt       


#        df = df[df['col1'].isin([2]) != True]
#        print(df_final)

#        print(df_liquid['Company Name'].nunique())

        datalake.save_transform_parquet('wood-mackenzie--corporate-analysis-aggregated-reported-production--transform', df_final, output_file)

print ('\n' f'COUNT: {x}')
       #Save new file
#    print(gas_file + str(13))

#    print(gas_file + str(14))
    # If we have updated any files, publish to Lens Direct
#publisher = DPCPublishing()
#publisher.publish_to_internal_lens_direct('wood-mackenzie--corporate-analysis-aggregated-reported-production--transform')   
